In [4]:
import glob
import pandas as pd
import numpy as np
import math

# Also needed:
# ipykernel (for running Jupyter Notebooks)
# openpyxl (to open Excel (.xlsx) files)

In [5]:
# Loading data

poc_files = glob.glob("temp_data/*.xlsx") # glob is used to generate a list of files matching an expression in a folder
poc_df = [pd.read_excel(file, sheet_name='Data') for file in poc_files] # we then loop through the list made (poc_files) reading each file
poc_data = pd.concat(poc_df) # then we compress this into a single dataframe (assuming consistent headers)

msc_poc_data = poc_data[poc_data['SampleType']=='MSC'] # extract just data labelled as 'MSC', marine snow catcher

### Main
Processing is done following the calculations presented in Giering et al 2023 $^{1}$. Changes are made to the equation for calculating P $_{fast}$, the result is divided by 1000, see
$$
P_{fast} = \frac{(P_{tray} - P_{base})\ ✕\ V_{tray} / (A_{tray}\ ✕\ h_{MSC})}{1000}
$$

to translate units to $\mu{g}\ L^{-1}$ rather than $\mu{g}\ m^{-1}$.

[1]: https://doi.org/10.1016/j.dsr2.2023.105277. S.L.C. Giering, R. Sanders, S. Blackbird, N. Briggs, F. Carvalho, H. East, B. Espinola, S.A. Henson, K. Kiriakoulakis, M.H. Iversen, R.S. Lampitt, K. Pabortsava, C. Pebody, K. Peel, C. Preece, K. Saw, M. Villa-Alfageme, G.A. Wolff,
Vertical imbalance in organic carbon budgets is indicative of a missing vertical transfer during a phytoplankton bloom near South Georgia (COMICS),
Deep Sea Research Part II: Topical Studies in Oceanography

In [6]:

# Volume of MSC tray (fixed for time being).
xx_vol_of_tray = 1

# General info
MSC_number      = []
MSC_Fraction    = []
MSC_POC_mean    = []
MSC_POC_std_err = []

# Particle fraction OC content
MSC_P_sus        = []
MSC_P_sus_err    = []

MSC_P_slow       = []
MSC_P_slow_err   = []

MSC_P_fast       = []
MSC_P_fast_err   = []

# Flux
MSC_F_slow       = []
MSC_F_slow_err   = []

MSC_F_fast_lower     = []
MSC_F_fast_lower_err = []
MSC_F_fast_upper     = []
MSC_F_fast_upper_err = []

# Main processing loop

for xx_MSC_number in pd.unique(msc_poc_data['CTD_UW_MSC_number']): # Work through each MSC.

    xx_cur_MSC_df = msc_poc_data[msc_poc_data['CTD_UW_MSC_number']==xx_MSC_number] # Separate dataframe for each MSC.

    # Initialise a bunch of blank lists to fill.
    # Fraction key.
    cur_frac = []

    # MSC fraction POC content.
    xx_cur_MSC_T0   = []
    xx_cur_MSC_Top  = []
    xx_cur_MSC_Base = []
    xx_cur_MSC_Tray = []

    # Particle fraction OC content.
    xx_cur_MSC_P_sus  = []
    xx_cur_MSC_P_slow = []
    xx_cur_MSC_P_fast = []

    xx_cur_MSC_P_sus_err  = []
    xx_cur_MSC_P_slow_err = []
    xx_cur_MSC_P_fast_err = []

    # Flux from each fraction.
    xx_cur_MSC_F_slow       = []
    xx_cur_MSC_F_fast_lower = []
    xx_cur_MSC_F_fast_upper = []

    xx_cur_MSC_F_slow_err       = []
    xx_cur_MSC_F_fast_lower_err = []
    xx_cur_MSC_F_fast_upper_err = []

    for xx_cur_frac in xx_cur_MSC_df['SampleSubtype']: # Loop through each MSC fraction.

        if xx_cur_frac in cur_frac: # If duplicated, skip, as above will catch both.

            continue

        else:

            cur_frac.append(xx_cur_frac)

            xx_cur_frac_df = xx_cur_MSC_df[xx_cur_MSC_df['SampleSubtype']==xx_cur_frac]

            xx_cur_frac_POC = np.mean(xx_cur_frac_df['POC_ug_L'])

            xx_cur_frac_POC_std = xx_cur_frac_df['POC_ug_L_stdev']

            ### Error propogation - This can/should be turned into a function
            xx_sum = 0
            for xx_POC_std in xx_cur_frac_POC_std:
                xx_sum+=xx_POC_std**2
                xx_cur_frac_POC_std_err = math.sqrt(xx_sum/(len(xx_cur_frac_POC_std)**2))
            ###
    
            if xx_cur_frac == 'Tzero':

                xx_cur_MSC_T0     = xx_cur_frac_POC
                xx_cur_MSC_T0_err = xx_cur_frac_POC_std_err

            elif xx_cur_frac == 'Top':

                xx_cur_MSC_Top     = xx_cur_frac_POC
                xx_cur_MSC_Top_err = xx_cur_frac_POC_std_err

            elif xx_cur_frac == 'Base':

                xx_cur_MSC_Base     = xx_cur_frac_POC
                xx_cur_MSC_Base_err = xx_cur_frac_POC_std_err

            elif xx_cur_frac == 'Tray':

                xx_cur_MSC_Tray     = xx_cur_frac_POC
                xx_cur_MSC_Tray_err = xx_cur_frac_POC_std_err

        MSC_number.append(xx_MSC_number)
        MSC_Fraction.append(xx_cur_frac)
        MSC_POC_mean.append(xx_cur_frac_POC)
        MSC_POC_std_err.append(xx_cur_frac_POC_std_err)

        ## Particle OC content

        ### 'Calculting' P_sus (assinging the Top fraction to P_sus!)
        if not xx_cur_MSC_Top:
            xx_cur_MSC_P_sus = []
            MSC_P_sus.append(xx_cur_MSC_P_sus)
        else:
            xx_cur_MSC_P_sus = xx_cur_MSC_Top
            MSC_P_sus.append(xx_cur_MSC_P_sus)

        #### 'Calculting' P_sus err
        if [x for x in (xx_cur_MSC_Top, xx_cur_MSC_Top_err) if not x]:
            xx_cur_MSC_P_sus_err = []
            MSC_P_sus_err.append(xx_cur_MSC_P_sus_err)
        else:
            xx_cur_MSC_P_sus_err = xx_cur_MSC_Top_err
            MSC_P_sus_err.append(xx_cur_MSC_P_sus_err)

        ### Calculting P_slow
        if [x for x in (xx_cur_MSC_Top, xx_cur_MSC_Base) if not x]:
            xx_cur_MSC_P_slow = []
            MSC_P_slow.append(xx_cur_MSC_P_slow)
        else:
            xx_cur_MSC_P_slow = ( (xx_cur_MSC_Base - xx_cur_MSC_Top) * 8/95 )
            MSC_P_slow.append(xx_cur_MSC_P_slow)

        #### Calculting P_slow err
        if [x for x in (xx_cur_MSC_Top, xx_cur_MSC_Base, xx_cur_MSC_Top_err, xx_cur_MSC_Base_err) if not x]:
            xx_cur_MSC_P_slow_err = []
            MSC_P_slow_err.append(xx_cur_MSC_P_slow_err)
        else:
            xx_cur_MSC_P_slow_err = ( (xx_cur_MSC_Base_err - xx_cur_MSC_Top_err) * 8/95 )
            MSC_P_slow_err.append(xx_cur_MSC_P_slow_err)

        ### Calculating P_fast
        if [x for x in (xx_cur_MSC_Tray, xx_cur_MSC_Base) if not x]:
            xx_cur_MSC_P_fast = []
            MSC_P_fast.append(xx_cur_MSC_P_fast)
        else:
            xx_cur_MSC_P_fast = ( ( (xx_cur_MSC_Tray - xx_cur_MSC_Base) * ( xx_vol_of_tray/(0.026/1.58) ) )/1000 )
            MSC_P_fast.append(xx_cur_MSC_P_fast)

        #### Calculating flux

        ### Calculating F_slow
        if not xx_cur_MSC_P_slow:
            xx_cur_MSC_F_slow = []
            MSC_F_slow.append(xx_cur_MSC_F_slow)
        else:
            xx_cur_MSC_F_slow = ( ( xx_cur_MSC_P_slow * 95 ) / (0.06 * (2/24)) )
            MSC_F_slow.append(xx_cur_MSC_F_slow)

        ### Calculating F_fast

            ### F_fast_lower
        if not xx_cur_MSC_P_fast:
            xx_cur_MSC_F_fast_lower = []
            MSC_F_fast_lower.append(xx_cur_MSC_F_fast_lower)
        else:
            xx_cur_MSC_F_fast_lower = ( ( xx_cur_MSC_P_fast * 95 ) / (0.06 * (2/24)) )
            MSC_F_fast_lower.append(xx_cur_MSC_F_fast_lower)
            
            ### F_fast_upper
        if not xx_cur_MSC_P_fast:
            xx_cur_MSC_F_fast_upper = []
            MSC_F_fast_upper.append(xx_cur_MSC_F_fast_upper)
        else:
            xx_cur_MSC_F_fast_upper = ( xx_cur_MSC_P_fast * 70 * 1000) # Multiplying by 1000 to go to µg/L
            MSC_F_fast_upper.append(xx_cur_MSC_F_fast_upper)


# Build into single dataframe

xx_all_data = {'MSC_number'       : MSC_number,
               'MSC_Fraction'     : MSC_Fraction, 
               'MSC_POC_mean'     : MSC_POC_mean, 
               'MSC_POC_std_err'  : MSC_POC_std_err,
               'MSC_P_sus'        : MSC_P_sus,
               'MSC_P_slow'       : MSC_P_slow,
               'MSC_P_fast'       : MSC_P_fast,
               'MSC_F_slow'       : MSC_F_slow,
               'MSC_F_fast_lower' : MSC_F_fast_lower,
               'MSC_F_fast_upper' : MSC_F_fast_upper
               }

MSC_POC = pd.DataFrame(xx_all_data)


In [ ]:
# Plotting


